---
description: This cookbook demonstate use of Langfuse with Azure OpenAI and Langchain for prompt versioning and evaluations 
---

# Langfuse integration with Azure OpenAI and Langchain  (Python)

Langfuse integrates with Langchain using the [Langchain Callbacks](https://python.langchain.com/docs/modules/callbacks/). Thereby, the Langfuse SDK automatically creates a nested trace for the abstractions offered by Langchain.

Add the handler as a callback when running your Langchain model/chain/agent:

```python /callbacks=[handler]/
# Initialize Langfuse handler
from langfuse.callback import CallbackHandler
handler = CallbackHandler(PUBLIC_KEY, SECRET_KEY)

# Setup Langchain
from langchain.chains import LLMChain
...
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

# Add Langfuse handler as callback
chain.run(input="<user_input", callbacks=[handler])
```

Langchain expression language (LCEL)
```python /config={"callbacks":[handler]}/
chain.invoke(input, config={"callbacks":[handler]})
```

---
 **_In case of missing events or tokens:_**

There are two ways to integrate callbacks into Langchain:
- *Constructor Callbacks*: Set when initializing an object, like `LLMChain(callbacks=[handler])` or `ChatOpenAI(callbacks=[handler])`. This approach will use the callback for every call made on that specific object. However, it won't apply to its child objects, making it limited in scope.
- *Request Callbacks*: Defined when issuing a request, like `chain.run(input, callbacks=[handler])` and `chain.invoke(input, config={"callbacks":[handler]})`. This not only uses the callback for that specific request but also for any subsequent sub-requests it triggers.

For comprehensive data capture especially for complex chains or agents, it's advised to use the both approaches, as demonstrated above [docs](https://python.langchain.com/docs/modules/callbacks/#where-to-pass-in-callbacks).

---

The Langfuse `CallbackHandler` tracks the following actions when using Langchain:

- Chains: `on_chain_start`, `on_chain_end`. `on_chain_error`
- Agents: `on_agent_start`, `on_agent_action`, `on_agent_finish`, `on_agent_end`
- Tools: `on_tool_start`, `on_tool_end`, `on_tool_error`
- Retriever: `on_retriever_start`, `on_retriever_end`
- ChatModel: `on_chat_model_start`,
- LLM: `on_llm_start`, `on_llm_end`, `on_llm_error`

Missing some useful information/context in Langfuse? Join the [Discord](/discord) or share your feedback directly with us: feedback@langfuse.com

## Notebook Setup

<NotebookBanner src="cookbook/integration_langchain.ipynb" />

### 1. Initializing the Langfuse Callback handler

The Langfuse SDKs are hosted on the pypi index.

In [ ]:
%pip install  --quiet langfuse langchain openai --upgrade
# After OpenAI release v1.0.0 in November 2023, you will need to install the langchain-openai package
%pip install --upgrade --quiet  langchain-openai

Initialize the client with api keys and optionally your environment. In the example we are using the cloud environment which is also the default.

Alternatively, you can also pass them as arguments to the `CallbackHandler` constructor.

In [ ]:
import os

# get keys for your project from https://cloud.langfuse.com
# get keys for your project from https://cloud.langfuse.com
os.environ[
    "LANGFUSE_HOST"
] = "https://cloud.langfuse.com"  # Your host, defaults to https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "Your public key"
os.environ["LANGFUSE_SECRET_KEY"] = "Your secret key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "your Azure OpenAI endpoint"
os.environ["AZURE_OPENAI_API_KEY"] = "your Azure OpenAI API key"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-09-01-preview"

In [ ]:
from langfuse.callback import CallbackHandler

handler = CallbackHandler()

In [ ]:
# handler is a callback handler that can be used to log and monitor the requests and responses.
handler.auth_check()

### 2. Langchain

In [ ]:
# further imports
# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage


from langfuse.callback import CallbackHandler

## Examples

In [ ]:
# First Example
from langfuse import Langfuse

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    model_name="gpt-3.5-turbo",
)

langfuse = Langfuse()

template = """
You are an AI assistant travel assistant that provides vacation recommendations to users. 
You should also be able to provide information about the weather, local customs, and travel restrictions. 
"""

# Create / update version of prompt in Langfuse
langfuse.create_prompt(
    name="travel_consultant",
    prompt=template,
    is_active=True,  # directly promote to production?
)


# Get version of prompt in Langfuse (cached for 5 minutes)
langfuse_prompt = langfuse.get_prompt("travel_consultant", cache_ttl_seconds=300).prompt


system_message_prompt = SystemMessagePromptTemplate.from_template(langfuse_prompt)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(
    f"Where should I go on vaction in Decemember for warm weather and beaches?",
    callbacks=[handler],
)

handler.flush()
print(result)

In [ ]:
handler.get_trace_url()

In [ ]:
# Second Example
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
import uuid

handler = CallbackHandler()


langfuse = Langfuse()

# Create custom name of trace which can help context of specific user_id, name and metadata

trace = langfuse.trace(name="chain_of_thought_example", user_id="user-1234")

handler = trace.get_langchain_handler()

prompt1 = ChatPromptTemplate.from_template(
    "What {type} is easiest to learn but hardest to master? Give a step by step approach of your thoughts, ending in your answer"
)
prompt2 = ChatPromptTemplate.from_template(
    "How {type} can be learned in 21 days? respond in {language}"
)

model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    model_name="gpt-3.5-turbo",
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"type": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke(
    {"type": "business", "language": "german"}, config={"callbacks": [handler]}
)

# multiple runs for one trace. This can be helpful to create scores for the different runs.
next_span_id = str(uuid.uuid4())
handler.setNextSpan(next_span_id)

chain2.invoke(
    {"type": "business", "language": "english"}, config={"callbacks": [handler]}
)
handler.get_trace_url()

## Adding scores

To add [scores](/docs/scores) to traces created with the Langchain integration, access the traceId via `handler.get_trace_id()`


### Example

In [ ]:
from langfuse import Langfuse


# Trace langchain run via the Langfuse CallbackHandler as shown above

# Get id of created trace
trace_id = handler.get_trace_id()

# Add score, e.g. via the Python SDK
langfuse = Langfuse()
trace = langfuse.score(
    trace_id=trace_id,
    name="user-explicit-feedback",
    value=1,
    comment="I like how personalized the response is",
)